## Objetivo

Modelar um classificador de alunos que prestaram o ENEM apenas para finalidade de treineiro.

In [242]:
import pandas as pd

In [243]:
%matplotlib inline  


In [244]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [245]:
print(train.shape, test.shape)

(13730, 167) (4570, 43)


## Análise dos dados

Baseado nas formas dos datasets, percebe-se que muitas features não estão presentes no set de teste. É esperado então que sejam trabalhadas apenas as colunas presentes no set de teste.

Como já discutido, o foco da análise é a variável `IN_TREINEIRO`

In [246]:
test.columns

Index(['NU_INSCRICAO', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA', 'NU_IDADE',
       'TP_SEXO', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC',
       'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DISLEXIA',
       'IN_DISCALCULIA', 'IN_SABATISTA', 'IN_GESTANTE', 'IN_IDOSO',
       'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT',
       'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'TP_LINGUA',
       'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3',
       'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002',
       'Q006', 'Q024', 'Q025', 'Q026', 'Q027', 'Q047'],
      dtype='object')

In [247]:
colunas = ['NU_INSCRICAO', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA', 'NU_IDADE',
       'TP_SEXO', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 
       'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT',
       'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'TP_LINGUA',
       'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3',
       'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 
       'Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047']

colunas = colunas + ['IN_TREINEIRO']

train = train[colunas]

In [248]:
train.IN_TREINEIRO.isna().sum()

0

In [249]:
train.IN_TREINEIRO.value_counts()

0    11947
1     1783
Name: IN_TREINEIRO, dtype: int64

In [250]:
train.corr().IN_TREINEIRO

CO_UF_RESIDENCIA   -0.034944
NU_IDADE           -0.295091
TP_COR_RACA         0.009676
TP_NACIONALIDADE   -0.015179
TP_ST_CONCLUSAO     0.533983
TP_ANO_CONCLUIU    -0.257710
TP_ESCOLA          -0.244562
TP_PRESENCA_CN      0.094692
TP_PRESENCA_CH      0.094692
TP_PRESENCA_LC      0.092454
TP_PRESENCA_MT      0.092454
NU_NOTA_CN         -0.037874
NU_NOTA_CH         -0.053460
NU_NOTA_LC         -0.028261
TP_LINGUA          -0.036395
NU_NOTA_COMP1      -0.008709
NU_NOTA_COMP2      -0.023308
NU_NOTA_COMP3      -0.030147
NU_NOTA_COMP4      -0.025958
NU_NOTA_COMP5      -0.025811
NU_NOTA_REDACAO    -0.026328
IN_TREINEIRO        1.000000
Name: IN_TREINEIRO, dtype: float64

Foram features com correlação medida maior ou menos que 0.2

In [251]:
fts_relevantes = ['NU_INSCRICAO', 'NU_IDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA']
target = ['IN_TREINEIRO']

In [252]:
train = train[fts_relevantes + target]
train.head()

,NU_INSCRICAO,NU_IDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO
0,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,24,1,4,1,0
1,2c3acac4b33ec2b195d77e7c04a2d75727fad723,17,2,0,2,0
2,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,21,3,0,1,0
3,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,25,1,9,1,0
4,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,28,1,4,1,0


Para que não haja inferência do modelo sobre os valores inteiros que aqui representam categorias, serão obtidas variáveis 'dummies'. 

In [253]:
dummies = pd.get_dummies(train[['TP_ST_CONCLUSAO', 'TP_ESCOLA']].astype(str))
train.drop(['TP_ST_CONCLUSAO', 'TP_ESCOLA'], axis=1, inplace=True)
train = pd.concat([train, dummies], axis=1, sort=False)

In [254]:
train.corr().IN_TREINEIRO

NU_IDADE            -0.295091
TP_ANO_CONCLUIU     -0.257710
IN_TREINEIRO         1.000000
TP_ST_CONCLUSAO_1   -0.367584
TP_ST_CONCLUSAO_2   -0.260120
TP_ST_CONCLUSAO_3    0.859856
TP_ST_CONCLUSAO_4   -0.083799
TP_ESCOLA_1          0.260120
TP_ESCOLA_2         -0.235201
TP_ESCOLA_3         -0.080326
TP_ESCOLA_4         -0.003297
Name: IN_TREINEIRO, dtype: float64

### Balanceamento do dataset com *undersampling*

Será selecionada uma amostra com a categoria mais numerosa.

In [255]:
df_train = pd.concat([train.query('IN_TREINEIRO == 1'), train.query('IN_TREINEIRO == 0').sample(1783, 
                        random_state=42)], sort=False)

In [256]:
df_train.IN_TREINEIRO.value_counts()

1    1783
0    1783
Name: IN_TREINEIRO, dtype: int64

In [257]:
y = df_train.IN_TREINEIRO
X = df_train.drop('IN_TREINEIRO', axis=1)

### Ajuste do set de teste

In [258]:
test = test[fts_relevantes]

In [259]:
dummies = pd.get_dummies(test[['TP_ST_CONCLUSAO', 'TP_ESCOLA']].astype(str))
test.drop(['TP_ST_CONCLUSAO', 'TP_ESCOLA'], axis=1, inplace=True)
test = pd.concat([test, dummies], axis=1, sort=False)

### AutoML

In [188]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))


Generation 1 - Current best internal CV score: 0.9913990689208582
Generation 2 - Current best internal CV score: 0.9913990689208582
Generation 3 - Current best internal CV score: 0.9913990689208582
Generation 4 - Current best internal CV score: 0.9913990689208582
Generation 5 - Current best internal CV score: 0.9913990689208582
Best pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.15000000000000002, min_samples_leaf=11, min_samples_split=13, n_estimators=100)
0.9966367713004485


In [264]:
predictions = tpot.predict(test.drop('NU_INSCRICAO', axis=1))

In [267]:
solution = pd.DataFrame({"NU_INSCRICAO": test.NU_INSCRICAO, "IN_TREINEIRO": predictions})
solution.to_csv("answer.csv", index = False)

In [268]:
solution

,NU_INSCRICAO,IN_TREINEIRO
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,0
1,177f281c68fa032aedbd842a745da68490926cd2,0
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,1
3,5c356d810fa57671402502cd0933e5601a2ebf1e,0
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,0
5,3f28749fb79fb059caf5aed79625a5addfd7a91a,0
6,bb2a0edddf3c59181a1496390aaaee7f32624d9d,1
7,cc7cab347fe5455aae983f3701ca40f84dc01949,0
8,95e9338f1da02f7bfa0e3194130afdccc0fb5457,1
9,155f84f2ee5b34e658f2adcc70f2ec83e37040cb,0
